In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 chromadb==0.4.12 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.33.0 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires pydantic>=2, but you have pydantic 1.10.18 which is incompatible.


In [2]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain import PromptTemplate, LLMChain

In [3]:
pip install -q accelerate bitsandbytes


Note: you may need to restart the kernel to use updated packages.


In [4]:
model_id = 'Manu1507/Llama-2-7b-finetune-summarizer'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [5]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

2024-08-26 15:12:56.812137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 15:12:56.812289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 15:12:56.958822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Prepare model, tokenizer: 423.413 sec.


In [6]:
time_1 = time()
summarization_pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.736 sec.


In [7]:
llm = HuggingFacePipeline(pipeline=summarization_pipeline, model_kwargs={'temperature': 0})

In [8]:
from langchain import PromptTemplate, LLMChain

template = """
Please summarize the following text in complete bullet points. Ensure that the summary covers all key points and is fully articulated:
{text}
BULLET POINT SUMMARY:
"""

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)


In [9]:
text = """ As part of Meta’s commitment to open science, today we are publicly releasing LLaMA (Large Language Model Meta AI), a state-of-the-art foundational large language model designed to help researchers advance their work in this subfield of AI. Smaller, more performant models such as LLaMA enable others in the research community who don’t have access to large amounts of infrastructure to study these models, further democratizing access in this important, fast-changing field.

Training smaller foundation models like LLaMA is desirable in the large language model space because it requires far less computing power and resources to test new approaches, validate others’ work, and explore new use cases. Foundation models train on a large set of unlabeled data, which makes them ideal for fine-tuning for a variety of tasks. We are making LLaMA available at several sizes (7B, 13B, 33B, and 65B parameters) and also sharing a LLaMA model card that details how we built the model in keeping with our approach to Responsible AI practices.

Over the last year, large language models — natural language processing (NLP) systems with billions of parameters — have shown new capabilities to generate creative text, solve mathematical theorems, predict protein structures, answer reading comprehension questions, and more. They are one of the clearest cases of the substantial potential benefits AI can offer at scale to billions of people.

Even with all the recent advancements in large language models, full research access to them remains limited because of the resources that are required to train and run such large models. This restricted access has limited researchers’ ability to understand how and why these large language models work, hindering progress on efforts to improve their robustness and mitigate known issues, such as bias, toxicity, and the potential for generating misinformation.

Smaller models trained on more tokens — which are pieces of words — are easier to retrain and fine-tune for specific potential product use cases. We trained LLaMA 65B and LLaMA 33B on 1.4 trillion tokens. Our smallest model, LLaMA 7B, is trained on one trillion tokens.

Like other large language models, LLaMA works by taking a sequence of words as an input and predicts a next word to recursively generate text. To train our model, we chose text from the 20 languages with the most speakers, focusing on those with Latin and Cyrillic alphabets.

There is still more research that needs to be done to address the risks of bias, toxic comments, and hallucinations in large language models. Like other models, LLaMA shares these challenges. As a foundation model, LLaMA is designed to be versatile and can be applied to many different use cases, versus a fine-tuned model that is designed for a specific task. By sharing the code for LLaMA, other researchers can more easily test new approaches to limiting or eliminating these problems in large language models. We also provide in the paper a set of evaluations on benchmarks evaluating model biases and toxicity to show the model’s limitations and to support further research in this crucial area.

To maintain integrity and prevent misuse, we are releasing our model under a noncommercial license focused on research use cases. Access to the model will be granted on a case-by-case basis to academic researchers; those affiliated with organizations in government, civil society, and academia; and industry research laboratories around the world. People interested in applying for access can find the link to the application in our research paper.

We believe that the entire AI community — academic researchers, civil society, policymakers, and industry — must work together to develop clear guidelines around responsible AI in general and responsible large language models in particular. We look forward to seeing what the community can learn — and eventually build — using LLaMA.
"""

In [10]:
print(llm_chain.run(text))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(



• Large language models are a fast-changing field that can offer significant benefits at scale to billions of people. 

• However, training these models requires significant computing power and resources, which limits access.

• Smaller foundation models, such as LLaMA, are easier to retrain and fine-tune for specific


In [11]:
text = """Artificial intelligence (AI) is a branch of computer science focused on developing intelligent machines that can mimic human cognitive 
           functions like learning, problem-solving, anddecision-making. AI research has made significant progress in recent years, with 
           advancementsin machine learning algorithms and the increasing availability of data. AI applications are nowwidespread, impacting various 
           sectors like healthcare, finance, transportation, and manufacturing.From medical diagnosis tools to self-driving cars, AI is transforming 
           industries and raisingethical considerations regarding automation, bias, and human control. As AI continues to evolve,it's crucial to 
           develop responsible AI practices that ensure its benefits are harnessed for thebetterment of society."""

In [12]:
print(llm_chain.run(text))

.	Introduction: AI has made significant progress in developing intelligent machines that can mimic human cognitive functions like learning and decision-making.
.	Advancement: AI research has made significant progress in recent years, with advancements in machine learning and the increasing availability of data.
.	Applications: AI applications are now widespread, impacting various sectors like healthcare, finance, transportation, and manufacturing.
.	Transforming industries: AI is transforming industries, raisingsome ethical considerations regarding automation, bias, and human control.
.	Harnessing: As AI continues to evolve, its crucial to develop responsible AI practices that ensure its benefits are for the betterment of society.


In [13]:
!pip install pyngrok flask_ngrok

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
!ngrok authtoken "YOUR AUTHTOKEN KEY"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [15]:
from pyngrok import ngrok

# Connect to the local Flask server running on port 5000
public_url = ngrok.connect(addr="5000")
print('* ngrok tunnel "{}" -> "http://127.0.0.1:5000"'.format(public_url))


* ngrok tunnel "NgrokTunnel: "https://37b2-104-196-176-56.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"


In [17]:
%mkdir templates -p

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text Summarization</title>
</head>
<body>
    <h1>Text Summarization</h1>
    <form action="/summarize" method="POST">
        <label for="input_text">Input Text:</label><br>
        <textarea id="input_text" name="input_text" rows="10" cols="80">{{ input_text }}</textarea><br><br>
        <input type="submit" value="Summarize">
    </form>
    {% if summary_text %}
    <h2>Summary:</h2>
    <textarea id="summary_text" rows="10" cols="80">{{ summary_text }}</textarea><br><br>
    <p>Generation Time: {{ generation_time }} seconds</p>
    {% endif %}
</body>
</html>

Writing templates/index.html


In [19]:
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import time

app = Flask(__name__)

def get_summary(text):
    try:
        start_time = time.time()
        result = llm_chain.run(text)
        print(f"Result type: {type(result)}")  # Print the type to verify
        print(f"Result content: {result}")  # Print the content to understand the output
        end_time = time.time()
        generation_time = round(end_time - start_time, 2)
        return result, generation_time
    except Exception as e:
        print(f"Exception: {e}")
        return f"An error occurred: {str(e)}", 0

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/summarize', methods=['POST'])
def summarize():
    try:
        input_text = request.form['input_text']
        summary, generation_time = get_summary(input_text)
        return render_template('index.html', input_text=input_text, summary_text=summary, generation_time=generation_time)
    except Exception as e:
        print(f"Error occurred: {e}")
        return render_template('index.html', input_text=input_text, summary_text="An error occurred.", generation_time=0)

# Get a public URL
public_url = ngrok.connect(5000)
print(f" * Running on {public_url}")

if __name__ == '__main__':
    app.run()

 * Running on NgrokTunnel: "https://fea9-104-196-176-56.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
Result type: <class 'str'>
Result content: 
* Artificial Intelligence (AI) focuses on creating machines capable of performing tasks that typically require human intelligence.
* Narrow AI simulates human cognition to perform a specific task while General AI would enable machines to perform any intellectual task.
* AI has significant implications for industries such as healthcare, finance, transportation, and entertainment.
* AI brings opportunities for innovation and improvement, however it also raises ethics and privacy concerns.
* The development of AI depends on how it is regulated and integrated into society.


In [22]:
!pip install flask_cors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import time

app = Flask(__name__)
CORS(app)  # Enable CORS

def get_summary(text):
    try:
        start_time = time.time()
        result = llm_chain.run(text)
        print(f"Result type: {type(result)}")
        print(f"Result content: {result}")
        end_time = time.time()
        generation_time = round(end_time - start_time, 2)
        return result, generation_time
    except Exception as e:
        print(f"Exception: {e}")
        return f"An error occurred: {str(e)}", 0

@app.route('/summarize', methods=['POST'])
def summarize():
    try:
        data = request.get_json()
        input_text = data.get('input_text', '')
        summary, generation_time = get_summary(input_text)
        return jsonify({
            'input_text': input_text,
            'summary_text': summary,
            'generation_time': generation_time
        })
    except Exception as e:
        print(f"Error occurred: {e}")
        return jsonify({
            'input_text': '',
            'summary_text': 'An error occurred.',
            'generation_time': 0
        })

# Get a public URL
public_url = ngrok.connect(5000)
print(f" * Running on {public_url}")

if __name__ == '__main__':
    app.run()


 * Running on NgrokTunnel: "https://c117-104-196-176-56.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
Result type: <class 'str'>
Result content: 
* AI is a field of computer science focused on creating machines that can perform tasks that typically need human intelligence. 
* AI is categorized into two types: Narrow AI and General AI. 
* Narrow AI focuses on specific tasks and operates within a limited context. 
* General AI would have the capacity to perform any intellectual task that a human can. 
* AI has profound implications across industries, including health, finance, transportation, and entertainment. 
* AI raises concerns about job displacement, privacy, and ethical implications. 
* The future of AI will depend on how it is developed and integrated into society.
AI is being developed in a wide range of fields, including: 
* Healthcare: AI is being used to diagnose diseases, develop personalized treatment plans, and predict patien